In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import seaborn as sns

In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import sys
sys.path.append('../src')


from fairsgod.losses import FairODLoss
from fairsgod.fairod import OutlierDetector
from fairsgod.fairod_sg import SGOutlierDetector
from fairsgod import evaluation
from sklearn.metrics import roc_auc_score

Init Plugin
Init Graph Optimizer
Init Kernel


In [4]:
SEED = 33
tf.random.set_seed(SEED)
np.random.seed(SEED)

In [5]:
# load adult
celeba_df = pd.read_csv('../datasets/proc/crafted_celeba.csv')
celeba_df = celeba_df.astype(np.float64)

y_pv = celeba_df[['Male','Attractive']]
X = celeba_df.drop(columns=['Male','Attractive'])

X_train, X_test, y_pv_train, y_pv_test = train_test_split(X, y_pv, test_size=0.3, shuffle=True)

pv_test = y_pv_test['Male']
pv_train = y_pv_train['Male']
y_train = y_pv_train['Attractive']
y_test = y_pv_test['Attractive']

FairOD on celebA dataset

In [10]:
model = OutlierDetector(alpha=0.1, gamma=0.8)
_, _ = model.fit(X_train, pv_train, batch_size=512, epochs=10, val_X=None, val_pv=None)

Predict

In [11]:
# predict
X_pred = model.predict_scores(X_test)

# compute reconstruction error of predicted samples and save in dataframe
#outlier_score = evalu.compute_outlier_scores(X_test, X_pred)
outlier_score = X_pred
pred_df = pd.DataFrame({'Reconstruction_error':outlier_score, 'True_class': y_test})

2021-12-18 16:11:25.460597: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


Evaluate

In [12]:
print("AUC score:   ", roc_auc_score(y_test, X_pred))
print("AUC RATIO:   ", evaluation.auc_ratio(y_test, X_pred, pv_test))
# AP ratio
print("AP RATIO ", evaluation.compute_AP_ratio(y_test, outlier_score, pv_test))
# Precision ratio
print("PRECISION RATIO ", evaluation.compute_precision_ratio(y_test, outlier_score, pv_test))
# Fairness metric
print("FAIRNESS METRIC  ", evaluation.compute_Fairness_metric(y_test, outlier_score, pv_test).numpy())
# Group Fidelity metric
print("GROUP FIDELITY METRIC  ", evaluation.compute_GF_metric(outlier_score, pv_test).numpy())

AUC score:    0.3047231376847986
AUC RATIO:    1.0657069690614946
AP RATIO  1.0128498674173785
PRECISION RATIO  1.0
FAIRNESS METRIC   0.024382081932511516
GROUP FIDELITY METRIC   1.0791788


## SG-AE Method

In [13]:
from fairsgod.losses import FairODLoss
from fairsgod.fairod import OutlierDetector
from fairsgod.fairod_sg import SGOutlierDetector
from fairsgod import evaluation

In [23]:
SEED = 33
tf.random.set_seed(SEED)
np.random.seed(SEED)

In [24]:
model = SGOutlierDetector(epsilon_p=0.001, 
                          lambda_se=4.86, 
                          lambda_a=25.0,
                          a=20, 
                          alpha=0.01, 
                          gamma=0.8)


_, _ = model.fit(X_train, pv_train, batch_size=512, epochs=10, val_X=None, val_pv=None)

EPOCH 0
  LOSS_TOTAL 2430.250732421875 - LOSS_REC 0.24965111911296844 - LOSS_SCORING 500.00018310546875 - EPSILON NORM 1.7319233417510986
  LOSS_TOTAL 2425.48828125 - LOSS_REC 0.25159862637519836 - LOSS_SCORING 499.0198669433594 - EPSILON NORM 1.7319233417510986
  LOSS_TOTAL 2415.97607421875 - LOSS_REC 0.2502082586288452 - LOSS_SCORING 497.06292724609375 - EPSILON NORM 1.7319233417510986
  LOSS_TOTAL 2415.956787109375 - LOSS_REC 0.24629950523376465 - LOSS_SCORING 497.0597839355469 - EPSILON NORM 1.7319233417510986
  LOSS_TOTAL 2406.45166015625 - LOSS_REC 0.251794695854187 - LOSS_SCORING 495.10284423828125 - EPSILON NORM 1.7319233417510986
  LOSS_TOTAL 2392.194580078125 - LOSS_REC 0.250404417514801 - LOSS_SCORING 492.1695556640625 - EPSILON NORM 1.7319233417510986
  LOSS_TOTAL 2411.1640625 - LOSS_REC 0.2541980743408203 - LOSS_SCORING 496.0719909667969 - EPSILON NORM 1.7319233417510986
  LOSS_TOTAL 2401.64794921875 - LOSS_REC 0.2461884319782257 - LOSS_SCORING 494.11553955078125 - EPSILON

In [29]:
# save predictions
X_pred = model.predict_scores(X_test).numpy()

In [28]:
print("AUC score:   ", roc_auc_score(y_test, X_pred))
# AUC ratio
print("AUC RATIO:   ", evaluation.auc_ratio(y_test, X_pred, pv_test))
# AP ratio
print("AP RATIO ", evaluation.compute_AP_ratio(y_test, outlier_score, pv_test))
# Precision ratio
print("PRECISION RATIO ", evaluation.compute_precision_ratio(y_test, outlier_score, pv_test))
# Fairness metric
print("FAIRNESS METRIC  ", evaluation.compute_Fairness_metric(y_test, outlier_score, pv_test).numpy())
# Group Fidelity metric
print("GROUP FIDELITY METRIC  ", evaluation.compute_GF_metric(outlier_score, pv_test).numpy())

AUC score:    0.6167790958274846
AUC RATIO:    0.8206148853941245
AP RATIO  1.0128498674173785
PRECISION RATIO  1.0
FAIRNESS METRIC   0.024382081932511516
GROUP FIDELITY METRIC   1.0791788
